In [50]:
import torch
import numpy as np
from  tqdm import tqdm


In [51]:
with open("../Data/mobydick.txt","r",encoding='utf-8') as file:
    raw_txt = file.read()

In [52]:
# Not beautiful
stripped_txt = raw_txt.replace('\n\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('\n', ' ')\
    .replace(';', '').replace(':', '').replace('  ', ' ')

In [53]:
vocab = list(set(stripped_txt))
vocab.sort()
vocab_dict = {}
for i, character in enumerate(vocab):
    identity_vector = torch.zeros(len(vocab))
    identity_vector[i] = 1
    vocab_dict[character] = identity_vector

In [54]:
# L
L = 15
dataX = torch.zeros((len(stripped_txt) - L, L, len(vocab_dict)))
dataY = torch.zeros(len(stripped_txt) - L)
for i in range(len(stripped_txt) - L):
    x = torch.zeros((L, len(vocab_dict)))
    for j in range(L):
        x[j] = vocab_dict[stripped_txt[i+j]]
    dataX[i] = x
    dataY[i] = torch.argmax(vocab_dict[stripped_txt[i+L]])

In [55]:
reversed_dict = {}
for key in vocab_dict:
    reversed_dict[torch.argmax(vocab_dict[key]).item()] = key
for x in dataX[0]:
    print(reversed_dict[torch.argmax(x).item()])
    print(x)
print(reversed_dict[dataY[0].item()])
print(dataY[0])

﻿
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.])
C
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
H
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,


In [56]:
from torch.utils.data import Dataset


class MobyDickDataset(Dataset):

    def __init__(self, x, y):
        self.X = x
        self.Y = y

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return (x, y)

    def __len__(self):
        return self.X.shape[0]

In [58]:
dataset = MobyDickDataset(dataX, dataY)
print(len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [1190000, 13171])

1203171


In [59]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=256, shuffle=True, num_workers=0)

In [60]:
class MyLSTM(torch.nn.Module):


    def __init__(self, feature_size, hidden_size, nr_of_stacks, output_dim):
        super().__init__()
        self.LSTM = torch.nn.LSTM(feature_size, hidden_size, nr_of_stacks, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_dim)
        self.hidden_size = hidden_size

    def forward(self, x):
        out, (h, c) = self.LSTM(x)
        out = self.linear(h)
        out = out[0]
        return out

In [61]:
import datetime

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device {device}.")

def train(model, optimizer, loss_f, epochs, train_loader):
    model.train()
    for epoch in range(epochs):
        loss_train = 0.0
        train_bar = tqdm(total=len(train_loader), desc="Epoch progression")

        for xb, yb in train_loader:
            xb = xb.to(device=device)
            yb = yb.to(device=device)
            yb = yb.long()
            with torch.cuda.amp.autocast(enabled=True):
                out = model(xb)
                loss = loss_f(out, yb)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss_train += loss.item()
                train_bar.update()

        print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))
        train_bar.close()

Device cuda.


In [62]:
model = MyLSTM(len(vocab_dict), len(vocab_dict), 1, len(vocab_dict))
opt = torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()
model.to(device=device)
loss.to(device=device)
train(model, opt, loss, 55, train_loader)

Epoch progression:   1%|          | 64/9297 [00:00<00:28, 327.59it/s]

2021-04-27 21:01:22.383899  |  Epoch 0  |  Training loss 2.151


Epoch progression:   1%|          | 67/9297 [00:00<00:27, 339.37it/s]

2021-04-27 21:01:46.046892  |  Epoch 1  |  Training loss 1.814


Epoch progression:   0%|          | 27/9297 [00:00<00:34, 269.76it/s]

...

2021-04-27 21:21:53.693907  |  Epoch 52  |  Training loss 1.391


Epoch progression:   0%|          | 28/9297 [00:00<00:33, 279.75it/s]

2021-04-27 21:22:17.419265  |  Epoch 53  |  Training loss 1.390


Epoch progression: 100%|██████████| 9297/9297 [00:23<00:00, 390.38it/s]

2021-04-27 21:22:41.234231  |  Epoch 54  |  Training loss 1.390


In [63]:
def validate(model, val_loader, train_loader):
    """
    Plot training and validation accuracy
    """
    model.eval()
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)

                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

In [64]:
validate(model, val_loader, train_loader)

Accuracy train: 0.58
Accuracy val: 0.56


{'train': 0.5793831932773109, 'val': 0.5588034317819451}

In [73]:
def generate_text(model, seed):
    seed = seed.to(device)
    for i, t in enumerate(seq_seed_tensor[0]):
        print(reversed_dict[torch.argmax(t).item()], end='')

    for _ in range(500):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0,:-1] = seed[0,1:].clone()
        seed[0, -1] = vocab_dict[reversed_dict[torch.argmax(next_c).item()]]
        print(reversed_dict[torch.argmax(next_c).item()], end='')

seq_seed = "There now is yo"

seq_seed_tensor = torch.zeros((1, L, len(vocab_dict)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab_dict[c]


for i, t in enumerate(seq_seed_tensor[0]):
    print(reversed_dict[torch.argmax(t).item()], end='')

There now is yo

In [66]:
generate_text(model, seq_seed_tensor)

There now      es of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of the strange of

It repeats!

In [74]:
def generate_text_with_sampling(model, seed):
    seed = seed.to(device)
    for i, t in enumerate(seq_seed_tensor[0]):
        print(reversed_dict[torch.argmax(t).item()], end='')

    for _ in range(1000):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0,:-1] = seed[0,1:].clone()
        test_c = np.random.choice(len(vocab_dict), 1, p=next_c.cpu().detach().numpy())
        seed[0, -1] = vocab_dict[reversed_dict[test_c[0]]]
        print(reversed_dict[test_c[0]], end='')

In [75]:
generate_text_with_sampling(model, seq_seed_tensor)

There now is your swiftly dood-whale-seclman days in their faith as I looked forward makes him, his crew, was surprips against their possiblating in the whale and Plate, on all these ambergrawhed in fight tun, you largery of wheeling and, and a farningly so war dost or Headed when them, and as the joms, not old about among himself, is species a shuttering, ‘not blanks were terror yourpent in a fire a man, in rust begins, the summit—but they dreary, comanting to this has being a mates,” smeatic in that all the myself in the zark suppered—he parts, second heaverrance, was strange cape and weaul upwards raped old concent the deame from eateral racks, and aghe in this old man care-no there’s Man!” Pullants and below various drop away—pall. Coming then, than snappe the hours in persuags are feit, of broken of look to air were a lone papur wrinkly arm shroud, yon reap now, londerable. For slouch-devilizes them, was hide! is no suids, for a resideracce lass, but maning and stood, must can be 

In [69]:
train(model, opt, loss, 25, train_loader)

Epoch progression:   1%|          | 66/9297 [00:00<00:27, 335.33it/s]

2021-04-27 21:23:17.174325  |  Epoch 0  |  Training loss 1.389


Epoch progression:   0%|          | 28/9297 [00:00<00:33, 279.77it/s]

2021-04-27 21:23:40.940638  |  Epoch 1  |  Training loss 1.388


Epoch progression:   0%|          | 23/9297 [00:00<00:40, 227.52it/s]

...

2021-04-27 21:31:58.895397  |  Epoch 22  |  Training loss 1.379


Epoch progression:   1%|          | 64/9297 [00:00<00:27, 330.29it/s]

2021-04-27 21:32:22.572409  |  Epoch 23  |  Training loss 1.379


Epoch progression: 100%|██████████| 9297/9297 [00:23<00:00, 389.67it/s]

2021-04-27 21:32:46.430812  |  Epoch 24  |  Training loss 1.379


In [72]:
seq_seed = "Call me Ishmael"

seq_seed_tensor = torch.zeros((1, L, len(vocab_dict)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab_dict[c]

generate_text_with_sampling(model, seq_seed_tensor)

Call me Ishmaelly so is hained his age he much a-pidest slide about veaully drivend at the unompleable and mentioned as out be something, an indivive battort seen in certain fore showerdable feet declaced to combined and scraping them nears. Though, the Sperm Whales in the sanctest inchernters consece than they rushed the deep in this vilially us to othermow compot this way, Hip then, as innerroniay whole even or one said in heavenly is turning the such energy, as queer through but pretening! What and if higher shout. But alwalloa his virguind replenis of the books of Englanderous lance is over the Ligath all it plane of Prowlinets. That thy time. And is marked Steelkilt but all who man yet?” “Thy heared the whale-kings huilied here? Out was that Eplances. Nower a things and backle my lass of the voyage boltss tallimple days poor tache, for, what is on zoling his blander, the clisked Whaling boat’s, and their didn’t reversed to see eye echant off from the deserted pair him the ship, ex

In [71]:
validate(model, val_loader, train_loader)



Accuracy train: 0.58
Accuracy val: 0.56


{'train': 0.5823983193277311, 'val': 0.5575127173335358}